In [24]:
# pip install cassandra-driver
# pip install astrapy

#To install 

#pip install tools

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
#from tools import showDF,examineCassandraTable


In [25]:
import json
astraKeyspace="killrvideo"

In [26]:
with open("killrvideocluster-token.json") as f:
    secrets = json.load(f)

In [27]:
CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

In [28]:
cloud_config= {
  'secure_connect_bundle': 'secure-connect-killrvideocluster.zip'
}

In [47]:
auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [32]:
rows=session.execute(" SELECT key,cluster_name,data_center FROM system.local;")
local=rows.one()
if local:
    print(" ** connected to cluster \ %s\ at data center \'%s\' ** " % (
    local.cluster_name,
    local.data_center,
    ))
else:
    print("Error could not read \ 'system.local\' table !")

 ** connected to cluster \ cndb\ at data center 'us-east1' ** 


In [104]:
import pandas as pd
# Query Data
query = f"SELECT * FROM {astraKeyspace}.users_by_city;"
result = session.execute(query)

# Convert result to DataFrame (optional)
df = pd.DataFrame(result.all())
df.head()


,city,last_name,first_name,email,address
0,Allapuzha,Chan,Jackie,jackiechan@kunfu.com,None
1,Allapuzha,Shon,Jackie,jackiechan@cobra.com,None
2,Allapuzha,Shon,Jackie,jackien@cobra.com,None
3,Kottayam,Doe,John,johndoe@gmail.com,Jinkx complex
4,Kolakata,john,Thomas,Joker,None


In [38]:
# Insert Data
query = "INSERT INTO killrvideo.users_by_city (city,last_name,first_name,email) VALUES ('Allapuzha', 'Shon','Jackie','jackien@cobra.com');"
prepared = session.prepare(query)

session.execute(query)


In [105]:
def insert_single(keyspace,table,data):
    cql = f"INSERT INTO {keyspace}.{table} ({','.join(data.keys())}) VALUES {tuple(i for i in data.values())};"
    try:
        prepared = session.prepare(cql)
        session.execute(prepared)
    except Exception as e:
        print(e)
    return cql
        


In [103]:
insert_single("killrvideo","users_by_city",{"last_name":'john',"email":"Joker","city":"Kolakata","first_name":"Thomas"})

"INSERT INTO killrvideo.users_by_city (last_name,email,city,first_name) VALUES ('john', 'Joker', 'Kolakata', 'Thomas');"

In [118]:
#q=create table killrvideo.customers(id int primary key ,name text ,age int );
def create(keyspace,table_name,columns,primary_key):
    cql=f"create table {keyspace}.{table_name}  ({",".join([f"{col} {val}"for col and val])})"
    return cql
    

In [119]:
create("shakila","call_list",{"id":"int","name":"text"},"id")

TypeError: sequence item 0: expected str instance, tuple found

In [107]:
class CassandraDB():
    def __init__(self,json_file_location,cloud_config):
        with open(json_file_location) as f:
            secrets = json.load(f)
        self.CLIENT_ID = secrets["clientId"]
        self.CLIENT_SECRET = secrets["secret"]
        auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
        cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
    def connect(self):
        session = cluster.connect()
        rows=session.execute(" SELECT key,cluster_name,data_center FROM system.local;")
        local=rows.one()
        if local:
            print(" ** connected to cluster \ %s\ at data center \'%s\' ** " % (
            local.cluster_name,
            local.data_center,
            ))
        else:
            print("Error could not read \ 'system.local\' table !")
    def insert_single(self,keyspace,table,data):
        session = cluster.connect()
        cql = f"INSERT INTO {keyspace}.{table} ({','.join(data.keys())}) VALUES {tuple(i for i in data.values())};"
        try:
            prepared = session.prepare(cql)
            session.execute(prepared)
        except Exception as e:
            print(e)
        finally:
            return cql
    def create_table()
        




In [ ]:
def insert_single(keyspace,column_name,values:list):
    cols=[]
    query = f"INSERT INTO {keyspace}.users_by_city {(city,last_name,first_name,email)} VALUES ('Allapuzha', 'Chan','Jackie','jackiechan@kunfu.com');"
    prepared = session.prepare(query)
    session.execute(prepared)

In [59]:

def create_record(driver, keyspace, table, data):
    """
  This function creates a new record in a Cassandra table.

  Args:
      driver: Cassandra driver object.
      keyspace: The keyspace where the table resides.
      table: The name of the Cassandra table.
      data: A dictionary containing the column names and their values for the new record.
    """
    session = driver.connect()
    try:
    # Build the CQL statement
        cql = f"INSERT INTO {keyspace}.{table} ({','.join(data.keys())}) VALUES ({','.join(data.keys())})"
        session.execute(cql)
        print(f"Record inserted successfully into {table} table.")
    except Exception as e:
        print(f"Error creating record: {e}")
    finally:
        session.shutdown()


In [60]:
create_record(cluster,'killrvideo','users_by_city',{"city":"Palakkad","email":"palakkadan@gmail.com"})

Error creating record: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:62 no viable alternative at input ',' (... (city,email) VALUES ([city],...)">
